<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/assignment_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題３

* Wikipediaの複数の記事を、lemmaを半角スペースでつないだ、長い文字列へ変換する。
 * ここでは、コンピュータ科学の様々な分野の記事を題材として使う。
* scikit-learnのCountVectorizerやTfidfVectorizerを使って、各記事における単語の出現頻度からなる文書ベクトルを得る。
* 特徴ベクトルどうしの類似度を計算し、「人工知能」分野と最も似ている順に　３つの分野がどの分野かを求める。
 * 答えは自分の感覚でチェック。
 * 文書ベクトルを作る時に、単語の品詞を名詞に限定するなど、品詞の情報を使うことで結果を改善できるかどうかも、余裕があれば試行錯誤する。

## spaCyのインストール

In [1]:
!pip install -U spacy

     |████████████████████████████████| 5.9 MB 5.0 MB/s 
     |████████████████████████████████| 10.1 MB 23.7 MB/s 
     |████████████████████████████████| 628 kB 61.0 MB/s 
     |████████████████████████████████| 451 kB 70.7 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [2]:
!python -m spacy download ja_core_news_sm

     |████████████████████████████████| 12.9 MB 197 kB/s 
     |████████████████████████████████| 86 kB 3.4 MB/s 
     |████████████████████████████████| 473 kB 35.1 MB/s 
  Created wheel for sudachidict-core: filename=SudachiDict_core-20210802-py3-none-any.whl size=71418512 sha256=a6b4ae56cd09b77d71ddb9bfd94228b0b3622ed016f687a4164a09bbe514d6d9
  Stored in directory: /root/.cache/pip/wheels/91/e8/21/e80d212743835d87bb5e7eca81b6abef6d8cb67a294007a837
  Created wheel for sudachipy: filename=SudachiPy-0.5.4-cp37-cp37m-linux_x86_64.whl size=872161 sha256=d22b52f57699d5a821f5532cf3ba4f54655578c220776f79a6f35aee7f197b5f
  Stored in directory: /root/.cache/pip/wheels/6b/5b/8b/ce1f543c9e9af590fdc62e8344fda5a3950c60c0d21c83174e
Successfully built sudachidict-core sudachipy
  Attempting uninstall: sortedcontainers
    Found existing installation: sortedcontainers 2.4.0
    Uninstalling sortedcontainers-2.4.0:
      Successfully uninstalled sortedcontainers-2.4.0
✔ Download and installation succ

* ここでランタイムを再起動する。その後、以下を実行。

In [1]:
from spacy.lang.ja import Japanese

nlp = Japanese()

## 日本語WIkipediaの人工知能の記事をダウンロード
* BeautifulSoupでHTMLのparsingを行う。

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

url = 'https://ja.wikipedia.org/wiki/%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

* 一連の処理を行う関数を定義しておく。
 * HTMLのpタグの中身を取り出し・・・
 * 形態素解析する。

In [3]:
def morph(soup, nlp):
  # 段落のテキストを取得する。
  lines = list()
  for para in soup.find_all('p'):
    lines.append(para.text)
  # Sudachiで形態素解析し、分かち書き後のlemmaを取得する。
  x_pos = ['SPACE', 'PUNCT', 'AUX', 'ADP', 'SYM', 'DET', 'SCONJ', 'PART']
  tokens = list()
  for line in lines:
    for token in nlp(line):
      pos = token.pos_
      if pos not in x_pos:
        tokens.append(token.lemma_)

  return ' '.join(tokens)

* 人工知能の記事だけ先に形態素解析する。

In [4]:
doc_AI = morph(soup, nlp)

## 比較対象の記事群を取得

In [5]:
target_str = '表話編歴コンピュータ科学'
prefix = '/wiki/'

urls = dict()
for table in soup.find_all('table'):
  if table.text[:len(target_str)] != target_str: continue
  for td in table.find_all('td'):
    for a in td.find_all('a'):
      if not a.text: continue
      try:
        if a.text.find('英語版') == -1:
          href = a['href']
          if href[:len(prefix)] == prefix and href.find('/Template:') == -1 and href.find('/Category:') == -1:
            urls[a.text] = 'https://ja.wikipedia.org' + href
      except:
        continue

for k in urls:
  print(k, urls[k])

プリント基板 https://ja.wikipedia.org/wiki/%E3%83%97%E3%83%AA%E3%83%B3%E3%83%88%E5%9F%BA%E6%9D%BF
周辺機器 https://ja.wikipedia.org/wiki/%E5%91%A8%E8%BE%BA%E6%A9%9F%E5%99%A8
集積回路 https://ja.wikipedia.org/wiki/%E9%9B%86%E7%A9%8D%E5%9B%9E%E8%B7%AF
Systems on Chip (SoCs) https://ja.wikipedia.org/wiki/System-on-a-chip
エネルギー消費 (グリーン・コンピューティング) https://ja.wikipedia.org/wiki/%E3%82%B0%E3%83%AA%E3%83%BC%E3%83%B3IT
EDA https://ja.wikipedia.org/wiki/EDA_(%E5%8D%8A%E5%B0%8E%E4%BD%93)
ハードウェアアクセラレーション https://ja.wikipedia.org/wiki/%E3%83%8F%E3%83%BC%E3%83%89%E3%82%A6%E3%82%A7%E3%82%A2%E3%82%A2%E3%82%AF%E3%82%BB%E3%83%A9%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3
コンピュータ・アーキテクチャ https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E3%83%BB%E3%82%A2%E3%83%BC%E3%82%AD%E3%83%86%E3%82%AF%E3%83%81%E3%83%A3
組み込みシステム https://ja.wikipedia.org/wiki/%E7%B5%84%E3%81%BF%E8%BE%BC%E3%81%BF%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0
リアルタイムシステム https://ja.wikipedia.org/wiki/%E3%83%AA%E3%82%A2%E3%83%

In [6]:
soups = dict()
for k in urls:
  html = urlopen(urls[k]) 
  soups[k] = BeautifulSoup(html, 'html.parser')

In [7]:
genre = list()
corpus = list()
for k in soups:
  genre.append(k)
  doc = morph(soups[k], nlp)
  corpus.append(doc)

* 先に処理しておいた人工知能の記事を含めてファイルに保存する。

In [8]:
import pandas as pd

genre.append('人工知能')
corpus.append(doc_AI)

df = pd.DataFrame(list(zip(genre, corpus)), columns=['genre', 'text'])
print(df.head())
df.to_csv('cs_corpus.csv')

                      genre                                               text
0                    プリント基板  プリント 基板 プリント きばる 短縮 形 pwb PCB 基板 一種 以下 ふた つ まと...
1                      周辺機器  周辺 機器 しゅうへん きく また ペリフェラル 英 peripheral コンピュータ ゲ...
2                      集積回路  集積 回路 しゅう せき かいろ 英 integrated circuit IC 半 導体 ...
3    Systems on Chip (SoCs)  System － on a chip SOC SOC 集積 回路 1 個 チップ 上 プロセ...
4  エネルギー消費 (グリーン・コンピューティング)  グリーン It グリーン アイティー 英 GREEN computing 地球 環境 配慮 ...


## CountVectorizerで単語の出現頻度を要素とする文書ベクトルを作成

In [12]:
import numpy as np
from scipy.spatial import distance
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('cs_corpus.csv')
genre = df['genre'].to_numpy()
index_AI = list(genres).index('人工知能')

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['text']).toarray()
print('文書数:{}, 語彙サイズ：{}'.format(*X.shape))
vocab = np.array(vectorizer.get_feature_names())
print('DFの降順: ' + ', '.join(list(vocab[np.argsort(- (X > 0).sum(0))[:10]])))
print()

top3 = np.argsort(np.linalg.norm(X - X[index_AI], axis=1))[1:4]
print('ユークリッド距離: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(- np.dot(X, X[index_AI]))[1:4]
print('内積: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(np.array([distance.cosine(x, X[index_AI]) for x in X]))[1:4]
print('コサイン類似度: ' + ', '.join(list(genre[top3])))

文書数:110, 語彙サイズ：9588
DFの降順: こと, する, ある, よる, いう, もの, ため, なる, また, おく

ユークリッド距離: オペレーションズ・リサーチ, 知識表現と推論, 計算幾何学
内積: オープンソースモデル, プログラミング言語, オペレーティングシステム
コサイン類似度: オペレーションズ・リサーチ, コンピュータビジョン, 知識表現と推論


## TfidfVectorizerでTF-IDF値を要素とする文書ベクトルを作成

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text']).toarray()
print('文書数:{}, 語彙サイズ：{}'.format(*X.shape))
vocab = np.array(vectorizer.get_feature_names())
print('DFの降順: ' + ', '.join(list(vocab[np.argsort(- (X > 0).sum(0))[:10]])))
print()

top3 = np.argsort(np.linalg.norm(X - X[index_AI], axis=1))[1:4]
print('ユークリッド距離: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(- np.dot(X, X[index_AI]))[1:4]
print('内積: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(np.array([distance.cosine(x, X[index_AI]) for x in X]))[1:4]
print('コサイン類似度: ' + ', '.join(list(genre[top3])))

文書数:110, 語彙サイズ：9588
DFの降順: こと, する, ある, よる, いう, もの, ため, なる, また, おく

ユークリッド距離: コンピュータビジョン, 計算社会科学, 自然言語処理
内積: コンピュータビジョン, 計算社会科学, 自然言語処理
コサイン類似度: コンピュータビジョン, 計算社会科学, 自然言語処理


## TfidfVectorizerのパラメータを変更してみる

In [16]:
vectorizer = TfidfVectorizer(min_df=2)
X = vectorizer.fit_transform(df['text']).toarray()
print('文書数:{}, 語彙サイズ：{}'.format(*X.shape))
vocab = np.array(vectorizer.get_feature_names())
print('DFの降順: ' + ', '.join(list(vocab[np.argsort(- (X > 0).sum(0))[:10]])))
print()

top3 = np.argsort(np.linalg.norm(X - X[index_AI], axis=1))[1:4]
print('ユークリッド距離: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(- np.dot(X, X[index_AI]))[1:4]
print('内積: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(np.array([distance.cosine(x, X[index_AI]) for x in X]))[1:4]
print('コサイン類似度: ' + ', '.join(list(genre[top3])))

文書数:110, 語彙サイズ：4109
DFの降順: こと, する, ある, よる, いう, もの, ため, なる, また, おく

ユークリッド距離: コンピュータビジョン, 計算社会科学, データマイニング
内積: コンピュータビジョン, 計算社会科学, データマイニング
コサイン類似度: コンピュータビジョン, 計算社会科学, データマイニング


In [17]:
vectorizer = TfidfVectorizer(max_df=.1)
X = vectorizer.fit_transform(df['text']).toarray()
print('文書数:{}, 語彙サイズ：{}'.format(*X.shape))
vocab = np.array(vectorizer.get_feature_names())
print('DFの降順: ' + ', '.join(list(vocab[np.argsort(- (X > 0).sum(0))[:10]])))
print()

top3 = np.argsort(np.linalg.norm(X - X[index_AI], axis=1))[1:4]
print('ユークリッド距離: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(- np.dot(X, X[index_AI]))[1:4]
print('内積: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(np.array([distance.cosine(x, X[index_AI]) for x in X]))[1:4]
print('コサイン類似度: ' + ', '.join(list(genre[top3])))

文書数:110, 語彙サイズ：8864
DFの降順: どれ, 実施, 起源, マーク, 意識, 表面, 顧客, 幾何, ところ, ネット

ユークリッド距離: 自然言語処理, 計算社会科学, コンピュータビジョン
内積: 自然言語処理, 計算社会科学, コンピュータビジョン
コサイン類似度: 自然言語処理, 計算社会科学, コンピュータビジョン


In [18]:
vectorizer = TfidfVectorizer(min_df=2, max_df=.1)
X = vectorizer.fit_transform(df['text']).toarray()
print('文書数:{}; 語彙サイズ：{}'.format(*X.shape))
vocab = np.array(vectorizer.get_feature_names())
print('DFの降順: ' + ', '.join(list(vocab[np.argsort(- (X > 0).sum(0))[:10]])))
print()

top3 = np.argsort(np.linalg.norm(X - X[index_AI], axis=1))[1:4]
print('ユークリッド距離: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(- np.dot(X, X[index_AI]))[1:4]
print('内積: ' + ', '.join(list(genre[top3])))
top3 = np.argsort(np.array([distance.cosine(x, X[index_AI]) for x in X]))[1:4]
print('コサイン類似度: ' + ', '.join(list(genre[top3])))

文書数:110; 語彙サイズ：3385
DFの降順: ネット, 離散, 静的, 背景, 導く, オンライン, マーク, 略称, メディア, 盛ん

ユークリッド距離: 自然言語処理, 計算社会科学, コンピュータビジョン
内積: 自然言語処理, 計算社会科学, コンピュータビジョン
コサイン類似度: 自然言語処理, 計算社会科学, コンピュータビジョン
